# Group Project: 数据预加载&清洗

<hr>

In [1]:
import os
import re
import torch
import json
import numpy as np
import pandas as pd
import urllib.request
import tarfile
from pathlib import Path
from typing import List, Dict, Tuple, Optional
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import html
from sklearn.model_selection import train_test_split

## DataDownloader - 数据下载与验证模块

**功能方法：**
- download_imdb() - 下载IMDb数据集压缩包并解压

- validate_data_directory() - 验证数据目录结构是否完整

- setup_data() - 主入口，确保数据可用（下载或使用现有数据）

<hr>

In [2]:
class DataDownloader:
    """数据下载与验证模块"""
    
    @staticmethod
    def download_imdb() -> Path:
        """下载IMDb数据集"""
        data_dir = Path('./aclImdb')
        
        if data_dir.exists():
            print("IMDb数据集已存在，跳过下载")
            return data_dir
        
        print("下载IMDb数据集...")
        url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
        tar_path = Path('./aclImdb_v1.tar.gz')
        
        try:
            urllib.request.urlretrieve(url, tar_path)
            print("下载完成，解压中...")
            
            with tarfile.open(tar_path, 'r:gz') as tar:
                tar.extractall(path='./')
            
            tar_path.unlink()
            print("IMDb数据集准备完成")
            return data_dir
        except Exception as e:
            raise Exception(f"数据集下载失败: {e}")
    
    @staticmethod
    def validate_data_directory(data_dir: Path) -> bool:
        """验证数据目录结构"""
        required_dirs = [
            data_dir / 'train' / 'pos',
            data_dir / 'train' / 'neg', 
            data_dir / 'test' / 'pos',
            data_dir / 'test' / 'neg'
        ]
        
        for dir_path in required_dirs:
            if not dir_path.exists():
                return False
            if len(list(dir_path.glob('*.txt'))) == 0:
                return False
                
        return True
    
    @staticmethod
    def setup_data() -> Path:
        """设置数据目录，确保数据可用"""
        data_dir = Path('./aclImdb')
        
        if DataDownloader.validate_data_directory(data_dir):
            print("✅ 数据目录验证通过")
            return data_dir
        
        print("数据目录不完整，重新下载...")
        data_dir = DataDownloader.download_imdb()
        
        if not DataDownloader.validate_data_directory(data_dir):
            raise Exception("数据目录验证失败，请手动检查数据集")
            
        return data_dir


## TextProcessor - 文本处理模块

**功能方法：**
- robust_text_cleaning(text) - 核心文本清洗，包含：

    1. HTML实体解码和标签移除
    
    2. URL处理
    
    3. 缩写词展开（如 "can't" → "can not"）
    
    4. 标点符号标准化
    
    5. 大小写统一

- process_in_batches() - 批量处理文本，避免内存溢出

<hr>

In [3]:
class TextProcessor:
    """文本处理模块"""
    
    @staticmethod
    def robust_text_cleaning(text: str) -> str:
        """鲁棒的文本清洗管道"""
        # 解码HTML实体
        text = html.unescape(text)
        
        # 移除HTML标签
        text = re.sub(r'<[^>]+>', ' ', text)
        
        # 处理URL
        text = re.sub(r'http\S+', ' <URL> ', text)
        
        # 处理常见的缩写和否定形式
        contractions = {
            r"won't": "will not", r"can't": "can not", r"n't": " not",
            r"'re": " are", r"'s": " is", r"'d": " would", 
            r"'ll": " will", r"'t": " not", r"'ve": " have",
            r"'m": " am"
        }
        
        for pattern, replacement in contractions.items():
            text = re.sub(pattern, replacement, text)
        
        # 保留基本的标点用于情感分析
        text = re.sub(r'[^a-zA-Z\s\.!?,;:\']', ' ', text)
        
        # 处理重复的标点
        text = re.sub(r'([!?.]){2,}', r'\1', text)
        text = re.sub(r'[!?.]+', r' \0 ', text)
        
        # 统一空格处理
        text = re.sub(r'\s+', ' ', text)
        
        return text.strip().lower()
    
    @staticmethod
    def process_in_batches(texts: List[str], process_func, batch_size: int = 1000, process_name: str = "处理") -> List[str]:
        """批量处理文本数据"""
        processed = []
        total = len(texts)
        
        for i in range(0, total, batch_size):
            batch = texts[i:i + batch_size]
            processed_batch = [process_func(text) for text in batch]
            processed.extend(processed_batch)
            
            if i % 5000 == 0 and i > 0:
                print(f"  {process_name} {i}/{total} 个样本")
        
        print(f"✅ {process_name}完成")
        return processed

## LabelProcessor - 标签处理模块

**功能方法：**

- parse_rating_from_filename() - 从文件名解析星级评分
- 
convert_to_binary() - 将星级转换为二分类
- get_label_schema_configig() - 获取标签
模式配置模式置

<hr>

In [4]:
class LabelProcessor:
    """标签处理模块 - 支持两种标签格式"""
    
    @staticmethod
    def parse_rating_from_filename(file_path: Path) -> Optional[int]:
        """从文件名解析星级评分"""
        stem = file_path.stem
        parts = stem.split('_')
        if len(parts) < 2:
            return None
        
        rating_str = parts[-1]
        try:
            return int(rating_str)  # 1..10
        except ValueError:
            return None
    
    @staticmethod
    def convert_to_binary(rating: int) -> int:
        """将星级转换为二分类标签"""
        if rating <= 4:
            return 0  # 负面
        else:  # rating >= 7
            return 1  # 正面
    
    @staticmethod
    def get_label_schema_config(schema_type: str) -> Dict:
        """获取标签模式配置"""
        schemas = {
            "binary": {
                "name": "二分类",
                "num_classes": 2,
                "output_dir": "processed_data_binary",
                "label_range": "0-1 (负面/正面)"
            },
            "multiclass": {
                "name": "多分类(1-10星)",
                "num_classes": 8,  # 实际只有8个类别 (1-4, 7-10)
                "output_dir": "processed_data_multiclass", 
                "label_range": "1-10星 (缺失5-6星)"
            }
        }
        return schemas.get(schema_type, schemas["binary"])

## DataLoaderManager - 数据加载管理器

**功能方法：**

- load_raw_data() - 根据标签模式加载原始IMDb数据
化格式
<hr>

In [5]:
class DataLoaderManager:
    """数据加载管理模块"""
    
    @staticmethod
    def load_raw_data(data_dir: Path, label_schema: str) -> Tuple[List[str], List[int], List[str], List[int]]:
        """加载原始IMDb数据，支持不同标签模式"""
        print(f"加载IMDb数据集 - {LabelProcessor.get_label_schema_config(label_schema)['name']}...")
        
        def load_from_directory(directory: Path, schema: str) -> Tuple[List[str], List[int]]:
            texts, labels = [], []
            
            for label_type in ['pos', 'neg']:
                dir_name = directory / label_type
                if not dir_name.exists():
                    print(f"Warning: 缺少目录: {dir_name}")
                    continue
                    
                for file_path in dir_name.glob('*.txt'):
                    # 解析星级评分
                    rating = LabelProcessor.parse_rating_from_filename(file_path)
                    if rating is None:
                        continue
                    
                    # 根据模式选择标签
                    if schema == "binary":
                        label = LabelProcessor.convert_to_binary(rating)
                    else:  # multiclass
                        label = rating
                    
                    # 读取文本
                    try:
                        with open(file_path, 'r', encoding='utf-8') as f:
                            text = f.read().strip()
                        texts.append(text)
                        labels.append(label)
                    except Exception as e:
                        print(f"跳过无法读取文件: {file_path.name} ({e})")
                        continue
            
            # 分布检查
            dist = Counter(labels)
            dist_sorted = dict(sorted(dist.items()))
            print(f"{directory.name} 标签分布: {dist_sorted}")
            return texts, labels
        
        train_dir = data_dir / 'train'
        test_dir = data_dir / 'test'
        
        train_texts, train_labels = load_from_directory(train_dir, label_schema)
        test_texts, test_labels = load_from_directory(test_dir, label_schema)
        
        print(f"加载了 {len(train_texts)} 训练样本, {len(test_texts)} 测试样本")
        return train_texts, train_labels, test_texts, test_labels

## DataSplitter - 数据划分器

**功能方法：**
- split_data_with_seeds()：使用随机种子进行可复现的数据划分
      1.  自动分层抽样保护    2. ─ 处理小样本类    3. ── 训练/验证/测试集划分(60%/20%/20    4. └── 分布统计输出

<hr>

In [6]:
class DataSplitter:
    """数据划分模块"""
    
    @staticmethod
    def split_data_with_seeds(texts: List[str], labels: List[int], seed: int = 42,
                             test_size: float = 0.2, val_size: float = 0.2) -> Tuple:
        """使用随机种子进行可复现的数据划分"""
        
        def _print_dist(name: str, y: List[int]):
            c = Counter(y)
            print(f"{name} 类别分布: {dict(sorted(c.items()))}")
        
        print("执行可复现的数据划分（带类别检查）...")
        counts = Counter(labels)
        min_count = min(counts.values()) if counts else 0
        use_stratify = labels if min_count >= 2 else None
        
        if use_stratify is None:
            print(f"Warning: 最小类别样本数 {min_count} < 2，初次划分不使用分层抽样")
        
        # 先划分测试集
        train_texts, test_texts, train_labels, test_labels = train_test_split(
            texts, labels, test_size=test_size, random_state=seed,
            stratify=use_stratify
        )
        
        # 再从训练集划分验证集
        train_counts = Counter(train_labels)
        min_train_count = min(train_counts.values()) if train_counts else 0
        use_stratify_val = train_labels if min_train_count >= 2 else None
        
        if use_stratify_val is None:
            print(f"Warning: 训练集中最小类别样本数 {min_train_count} < 2，验证集划分不使用分层抽样")
        
        train_texts, val_texts, train_labels, val_labels = train_test_split(
            train_texts, train_labels, test_size=val_size, random_state=seed,
            stratify=use_stratify_val
        )
        
        print(f"数据划分: 训练集={len(train_texts)}, 验证集={len(val_texts)}, 测试集={len(test_texts)}")
        _print_dist("训练集", train_labels)
        _print_dist("验证集", val_labels)
        _print_dist("测试集", test_labels)
        
        return train_texts, train_labels, val_texts, val_labels, test_texts, test_labels

## VocabularyBuilder - 词汇表管理模块

**功能方法：**
- build_vocabulary(texts) - 基于训练文本构建词汇表：

    1. 统计词频
    
    2. 过滤低频词
    
    3. 添加特殊标记（PAD/UNK/BOS/EOS）
    
    4. 计算词汇表覆盖率

- text_to_sequence(text) - 将文本转换为数字序列：

    1. 添加开始/结束标记
    
    2. 处理未知词（OOV）
    
    3. 序列填充/截断
    
    4. 生成注意力掩码

<hr>

In [7]:
class VocabularyBuilder:
    """词汇表管理模块"""
    
    def __init__(self, max_vocab_size: int = 30000, min_freq: int = 2):
        self.max_vocab_size = max_vocab_size
        self.min_freq = min_freq
        self.special_tokens = ['<PAD>', '<UNK>', '<BOS>', '<EOS>']
        self.token_to_id = {}
        self.id_to_token = {}
        self.vocab_size = 0
    
    def build_vocabulary(self, texts: List[str]) -> None:
        """构建词汇表"""
        print("构建词汇表...")
        
        # 统计词频
        word_freq = Counter()
        for text in texts:
            tokens = text.split()
            word_freq.update(tokens)
        
        # 过滤低频词
        filtered_words = [(word, freq) for word, freq in word_freq.items() 
                         if freq >= self.min_freq]
        
        # 按频率排序并选择前N个词
        sorted_words = sorted(filtered_words, key=lambda x: x[1], reverse=True)
        selected_words = [word for word, freq in sorted_words[:self.max_vocab_size - len(self.special_tokens)]]
        
        # 构建词汇表
        self.token_to_id = {}
        self.id_to_token = {}
        
        # 添加特殊标记
        for idx, token in enumerate(self.special_tokens):
            self.token_to_id[token] = idx
            self.id_to_token[idx] = token
        
        # 添加普通词汇
        for idx, word in enumerate(selected_words, start=len(self.special_tokens)):
            self.token_to_id[word] = idx
            self.id_to_token[idx] = word
        
        self.vocab_size = len(self.token_to_id)
        
        # OOV分析
        total_tokens = sum(word_freq.values())
        covered_tokens = sum(freq for word, freq in word_freq.items() 
                           if word in self.token_to_id)
        coverage = covered_tokens / total_tokens * 100
        
        print(f"词汇表构建完成: {self.vocab_size} 个token")
        print(f"词汇表覆盖率: {coverage:.2f}%")
    
    def text_to_sequence(self, text: str, max_length: int = 512, 
                        add_special_tokens: bool = True) -> Tuple[List[int], List[int], int]:
        """将文本转换为整数序列"""
        tokens = text.split()
        sequence = []
        
        # 添加开始标记
        if add_special_tokens:
            sequence.append(self.token_to_id['<BOS>'])
        
        # 转换tokens
        for token in tokens:
            sequence.append(self.token_to_id.get(token, self.token_to_id['<UNK>']))
        
        # 添加结束标记
        if add_special_tokens:
            sequence.append(self.token_to_id['<EOS>'])
        
        original_length = len(sequence)
        
        # 填充或截断
        if len(sequence) < max_length:
            sequence.extend([self.token_to_id['<PAD>']] * (max_length - len(sequence)))
            attention_mask = [1] * original_length + [0] * (max_length - original_length)
        else:
            sequence = sequence[:max_length]
            attention_mask = [1] * max_length
        
        return sequence, attention_mask, original_length

##  IMDBDataset - 数据集接口模块

**功能方法：**

- __init__() - 初始化数据集，验证数据一致性

- __len__() - 返回数据集大小

- __getitem__() - 获取单个样本，返回字典格式

<hr>

In [8]:
class IMDBDataset(Dataset):
    """IMDb数据集类"""
    
    def __init__(self, sequences: List[List[int]], attention_masks: List[List[int]], 
                 labels: List[int], lengths: List[int]):
        self.sequences = torch.tensor(sequences, dtype=torch.long)
        self.attention_masks = torch.tensor(attention_masks, dtype=torch.long)
        self.labels = torch.tensor(labels, dtype=torch.long)
        self.lengths = torch.tensor(lengths, dtype=torch.long)
        
        # 验证数据形状
        assert len(self.sequences) == len(self.labels), "序列和标签数量不匹配"
        assert len(self.sequences) == len(self.attention_masks), "序列和注意力掩码数量不匹配"
        assert len(self.sequences) == len(self.lengths), "序列和长度数量不匹配"
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return {
            'input_ids': self.sequences[idx],
            'attention_mask': self.attention_masks[idx],
            'labels': self.labels[idx],
            'lengths': self.lengths[idx]
        }

## DataSaver - 数据保存模块

**功能方法：**

- save_processed_data() - 主保存函数

- _save_vocabulary() - 保存词汇表为JSON

- _save_config() - 保存预处理配置

- _save_tensor_data() - 保存Tensor数据为.pt文件

- _save_metadata() - 保存统计元数据

- _verify_files() - 验证生成的文件

<hr>

In [9]:
class DataSaver:
    """数据保存模块"""
    
    @staticmethod
    def save_processed_data(datasets: Dict, vocab, metadata: Dict, output_dir: str, label_schema: str):
        """保存所有处理后的数据"""
        output_path = Path(output_dir)
        output_path.mkdir(parents=True, exist_ok=True)
        
        schema_config = LabelProcessor.get_label_schema_config(label_schema)
        print(f"开始保存{schema_config['name']}数据到: {output_path}")
        
        # 1. 保存词汇表
        DataSaver._save_vocabulary(vocab, output_path)
        
        # 2. 保存配置
        DataSaver._save_config(output_path, label_schema, schema_config)
        
        # 3. 保存 tensor 数据
        DataSaver._save_tensor_data(datasets, output_path)
        
        # 4. 保存元数据
        DataSaver._save_metadata(metadata, output_path)
        
        # 5. 验证文件
        DataSaver._verify_files(output_path)
    
    @staticmethod
    def _save_vocabulary(vocab, output_path):
        """保存词汇表"""
        vocab_path = output_path / 'vocabulary.json'
        with open(vocab_path, 'w', encoding='utf-8') as f:
            json.dump(vocab, f, indent=2, ensure_ascii=False)
        print(f"✅ 词汇表保存到: {vocab_path}")
    
    @staticmethod
    def _save_config(output_path, label_schema, schema_config):
        """保存配置"""
        config_path = output_path / 'preprocessing_config.json'
        config = {
            'label_schema': label_schema,
            'schema_name': schema_config['name'],
            'num_classes': schema_config['num_classes'],
            'label_range': schema_config['label_range'],
            'max_vocab_size': 30000,
            'max_length': 512,
            'min_freq': 2,
            'seed': 42,
            'saved_time': str(pd.Timestamp.now())
        }
        with open(config_path, 'w', encoding='utf-8') as f:
            json.dump(config, f, indent=2)
        print(f"✅ 配置保存到: {config_path}")
    
    @staticmethod
    def _save_tensor_data(datasets, output_path):
        """保存 tensor 数据"""
        data_path = output_path / 'all_data.pt'
        
        save_data = {
            'train_sequences': datasets['train'].sequences,
            'train_masks': datasets['train'].attention_masks,
            'train_labels': datasets['train'].labels,
            'train_lengths': datasets['train'].lengths,
            
            'val_sequences': datasets['val'].sequences,
            'val_masks': datasets['val'].attention_masks,
            'val_labels': datasets['val'].labels,
            'val_lengths': datasets['val'].lengths,
            
            'test_sequences': datasets['test'].sequences,
            'test_masks': datasets['test'].attention_masks,
            'test_labels': datasets['test'].labels,
            'test_lengths': datasets['test'].lengths,
        }
        
        torch.save(save_data, data_path)
        file_size = data_path.stat().st_size / 1024 / 1024
        print(f"✅ Tensor数据保存到: {data_path} (大小: {file_size:.2f} MB)")
    
    @staticmethod
    def _save_metadata(metadata, output_path):
        """保存元数据"""
        metadata_path = output_path / 'preprocessing_metadata.json'
        with open(metadata_path, 'w', encoding='utf-8') as f:
            json.dump(metadata, f, indent=2, default=str, ensure_ascii=False)
        print(f"✅ 元数据保存到: {metadata_path}")
    
    @staticmethod
    def _verify_files(output_path):
        """验证文件是否创建成功"""
        print("\n📁 验证生成的文件:")
        for file in output_path.iterdir():
            size_kb = file.stat().st_size / 1024
            print(f"  {file.name} ({size_kb:.1f} KB)")


## UnifiedDataProcessingPipeline - 流程协调模块

**功能方法：**

- run_complete_pipeline() - 执行完整的多模式数据处理流程

- process_single_schema() - 处理单个标签模式的数据

- _ensure_data_ready() - 确保数据准备就绪（只执行一次）

- _vectorize_texts() - 批量向量化文本

- _create_datasets() - 创建训练/验证/测试数据集

- _check_data_size() - 检查数据规模
<hr>

In [10]:
class UnifiedDataProcessingPipeline:
    """统一的数据处理流水线"""
    
    def __init__(self, max_vocab_size: int = 30000, max_length: int = 512, 
                 min_freq: int = 2, seed: int = 42):
        self.max_vocab_size = max_vocab_size
        self.max_length = max_length
        self.min_freq = min_freq
        self.seed = seed
        self.data_dir = None  # 添加数据目录缓存
    
    def _ensure_data_ready(self) -> bool:
        """确保数据准备就绪（只执行一次）"""
        if self.data_dir is not None:
            return True
            
        try:
            print("🔍 检查数据源...")
            self.data_dir = DataDownloader.setup_data()
            print("✅ 数据源准备完成")
            return True
        except Exception as e:
            print(f"❌ 数据源准备失败: {e}")
            return False
    
    def process_single_schema(self, label_schema: str) -> bool:
        """处理单个标签模式的数据"""
        try:
            # 0. 确保数据就绪
            if not self._ensure_data_ready():
                return False
                
            schema_config = LabelProcessor.get_label_schema_config(label_schema)
            print(f"\n{'='*50}")
            print(f"=== 处理 {schema_config['name']} 数据 ===")
            print(f"标签范围: {schema_config['label_range']}")
            print(f"类别数量: {schema_config['num_classes']}")
            print(f"输出目录: {schema_config['output_dir']}")
            print(f"{'='*50}")
            
            # 1. 初始化组件
            vocab_builder = VocabularyBuilder(self.max_vocab_size, self.min_freq)
            text_processor = TextProcessor()
            
            # 2. 加载原始数据
            print("\n📥 加载原始数据...")
            train_texts, train_labels, test_texts, test_labels = DataLoaderManager.load_raw_data(
                self.data_dir, label_schema
            )
            
            # 3. 合并并划分数据
            print("📊 数据划分...")
            all_texts = train_texts + test_texts
            all_labels = train_labels + test_labels
            
            train_texts, train_labels, val_texts, val_labels, test_texts, test_labels = \
                DataSplitter.split_data_with_seeds(all_texts, all_labels, self.seed)
            
            # 4. 文本清洗
            print("🧹 清洗文本数据...")
            train_texts_clean = text_processor.process_in_batches(
                train_texts, text_processor.robust_text_cleaning, process_name="文本清洗"
            )
            val_texts_clean = text_processor.process_in_batches(
                val_texts, text_processor.robust_text_cleaning, process_name="文本清洗"
            )
            test_texts_clean = text_processor.process_in_batches(
                test_texts, text_processor.robust_text_cleaning, process_name="文本清洗"
            )
            
            # 5. 构建词汇表
            print("📚 构建词汇表...")
            vocab_builder.build_vocabulary(train_texts_clean)
            
            # 6. 文本向量化
            print("🔢 向量化数据...")
            train_sequences, train_masks, train_lengths = self._vectorize_texts(
                train_texts_clean, vocab_builder, self.max_length
            )
            val_sequences, val_masks, val_lengths = self._vectorize_texts(
                val_texts_clean, vocab_builder, self.max_length
            )
            test_sequences, test_masks, test_lengths = self._vectorize_texts(
                test_texts_clean, vocab_builder, self.max_length
            )
            
            # 7. 创建数据集
            print("🗂️ 创建数据集...")
            datasets = self._create_datasets(
                train_sequences, train_masks, train_labels, train_lengths,
                val_sequences, val_masks, val_labels, val_lengths,
                test_sequences, test_masks, test_labels, test_lengths
            )
            
            # 8. 准备保存数据
            vocab_info = {
                'token_to_id': vocab_builder.token_to_id,
                'id_to_token': vocab_builder.id_to_token,
                'vocab_size': vocab_builder.vocab_size,
                'special_tokens': vocab_builder.special_tokens
            }
            
            metadata = {
                'label_schema': label_schema,
                'schema_config': schema_config,
                'sequence_length_analysis': {
                    'mean_length': np.mean(train_lengths),
                    'max_length': np.max(train_lengths),
                    'min_length': np.min(train_lengths),
                    'total_samples': len(train_sequences)
                },
                'class_distribution': {
                    'train': dict(Counter(train_labels)),
                    'val': dict(Counter(val_labels)),
                    'test': dict(Counter(test_labels))
                }
            }
            
            # 9. 保存数据
            DataSaver.save_processed_data(
                datasets, vocab_info, metadata, 
                f"./{schema_config['output_dir']}", 
                label_schema
            )
            
            print(f"🎉 {schema_config['name']}数据处理完成！")
            return True
            
        except Exception as e:
            print(f"❌ {label_schema}处理失败: {e}")
            import traceback
            traceback.print_exc()
            return False
    
    def run_complete_pipeline(self) -> bool:
        """运行完整的多模式数据处理流水线"""
        print("🚀 开始统一数据处理流水线")
        
        # 先统一检查数据源
        print("📦 准备数据源...")
        if not self._ensure_data_ready():
            print("❌ 数据源准备失败，终止处理")
            return False
            
        success_count = 0
        schemas = ["binary", "multiclass"]
        
        for schema in schemas:
            success = self.process_single_schema(schema)
            if success:
                success_count += 1
                self._check_data_size(schema)
        
        print(f"\n{'='*60}")
        print(f"🎯 数据处理完成总结:")
        print(f"成功处理: {success_count}/{len(schemas)} 种数据模式")
        
        if success_count == len(schemas):
            print("✅ 所有数据模式处理完成！")
            print("📁 生成的数据目录:")
            for schema in schemas:
                config = LabelProcessor.get_label_schema_config(schema)
                print(f"  - {config['output_dir']}: {config['name']}")
            return True
        else:
            print("❌ 部分数据处理失败")
            return False
    
    def _vectorize_texts(self, texts: List[str], vocab_builder: VocabularyBuilder, max_length: int):
        """向量化文本数据"""
        sequences, masks, lengths = [], [], []
        
        for i, text in enumerate(texts):
            seq, mask, length = vocab_builder.text_to_sequence(text, max_length)
            sequences.append(seq)
            masks.append(mask)
            lengths.append(length)
            
            if i % 5000 == 0 and i > 0:
                print(f"  已向量化 {i}/{len(texts)} 个样本")
        
        return sequences, masks, lengths
    
    def _create_datasets(self, train_sequences, train_masks, train_labels, train_lengths,
                        val_sequences, val_masks, val_labels, val_lengths,
                        test_sequences, test_masks, test_labels, test_lengths):
        """创建数据集"""
        train_dataset = IMDBDataset(train_sequences, train_masks, train_labels, train_lengths)
        val_dataset = IMDBDataset(val_sequences, val_masks, val_labels, val_lengths)
        test_dataset = IMDBDataset(test_sequences, test_masks, test_labels, test_lengths)
        
        return {
            'train': train_dataset,
            'val': val_dataset,
            'test': test_dataset
        }
    
    def _check_data_size(self, label_schema: str):
        """检查数据规模"""
        config = LabelProcessor.get_label_schema_config(label_schema)
        data_path = f"./{config['output_dir']}/all_data.pt"
        
        try:
            data = torch.load(data_path)
            
            print(f"\n=== {config['name']}数据规模检查 ===")
            print(f"训练序列: {data['train_sequences'].shape}")
            print(f"训练标签: {data['train_labels'].shape}")
            print(f"验证序列: {data['val_sequences'].shape}")
            print(f"测试序列: {data['test_sequences'].shape}")
            
            total_samples = (data['train_sequences'].shape[0] + 
                            data['val_sequences'].shape[0] + 
                            data['test_sequences'].shape[0])
            
            print(f"总样本数: {total_samples}")
            
            if total_samples >= 40000:
                print("✅ 处理了完整数据集")
            else:
                print("❌ 数据量异常，请检查处理过程")
                
        except Exception as e:
            print(f"检查失败: {e}")

## 工具函数

**功能方法：**

检查处理后的数据规模和统计信息

<hr>

In [11]:
def check_data_size(data_path: str = './processed_data/all_data.pt'):
    """检查处理了多少数据"""
    try:
        data = torch.load(data_path)
        
        print("\n=== 数据规模检查 ===")
        print(f"训练序列: {data['train_sequences'].shape}")
        print(f"训练标签: {data['train_labels'].shape}")
        print(f"验证序列: {data['val_sequences'].shape}") 
        print(f"测试序列: {data['test_sequences'].shape}")
        
        total_samples = (data['train_sequences'].shape[0] + 
                        data['val_sequences'].shape[0] + 
                        data['test_sequences'].shape[0])
        
        print(f"\n总样本数: {total_samples}")
        
        if total_samples >= 40000:
            print("✅ 处理了完整数据集")
        elif total_samples >= 20000:
            print("⚠️  处理了部分数据集")  
        else:
            print("❌ 数据量异常，请检查处理过程")
            
    except Exception as e:
        print(f"检查失败: {e}")

## 主程序入口

<hr>

In [12]:
# 主程序入口
if __name__ == "__main__":
    # 创建统一数据处理流水线
    pipeline = UnifiedDataProcessingPipeline(
        max_vocab_size=30000,
        max_length=512, 
        min_freq=2,
        seed=42
    )
    
    # 运行完整的多模式处理
    success = pipeline.run_complete_pipeline()
    
    if success:
        check_data_size('./processed_data_binary/all_data.pt')
        check_data_size('./processed_data_multiclass/all_data.pt')
        print("\n🎉 所有数据处理完成！现在可以开始训练不同任务的模型了！")
        print("   - 二分类情感分析: 使用 processed_data_binary/")
        print("   - 多分类星级预测: 使用 processed_data_multiclass/")
    else:
        print("\n❌ 数据处理失败，请检查错误信息")

🚀 开始统一数据处理流水线
📦 准备数据源...
🔍 检查数据源...
数据目录不完整，重新下载...
下载IMDb数据集...
下载完成，解压中...
IMDb数据集准备完成
✅ 数据源准备完成

=== 处理 二分类 数据 ===
标签范围: 0-1 (负面/正面)
类别数量: 2
输出目录: processed_data_binary

📥 加载原始数据...
加载IMDb数据集 - 二分类...
train 标签分布: {0: 12500, 1: 12500}
test 标签分布: {0: 12500, 1: 12500}
加载了 25000 训练样本, 25000 测试样本
📊 数据划分...
执行可复现的数据划分（带类别检查）...
数据划分: 训练集=32000, 验证集=8000, 测试集=10000
训练集 类别分布: {0: 16000, 1: 16000}
验证集 类别分布: {0: 4000, 1: 4000}
测试集 类别分布: {0: 5000, 1: 5000}
🧹 清洗文本数据...
  文本清洗 5000/32000 个样本
  文本清洗 10000/32000 个样本
  文本清洗 15000/32000 个样本
  文本清洗 20000/32000 个样本
  文本清洗 25000/32000 个样本
  文本清洗 30000/32000 个样本
✅ 文本清洗完成
  文本清洗 5000/8000 个样本
✅ 文本清洗完成
  文本清洗 5000/10000 个样本
✅ 文本清洗完成
📚 构建词汇表...
构建词汇表...
词汇表构建完成: 30000 个token
词汇表覆盖率: 97.52%
🔢 向量化数据...
  已向量化 5000/32000 个样本
  已向量化 10000/32000 个样本
  已向量化 15000/32000 个样本
  已向量化 20000/32000 个样本
  已向量化 25000/32000 个样本
  已向量化 30000/32000 个样本
  已向量化 5000/8000 个样本
  已向量化 5000/10000 个样本
🗂️ 创建数据集...
开始保存二分类数据到: processed_data_binary
✅ 词汇表保存到: processed_data_binary\vo

C:\Users\f5428489\AppData\Local\Temp\ipykernel_4412\2919244761.py:204: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(data_path)


train 标签分布: {1: 5100, 2: 2284, 3: 2420, 4: 2696, 7: 2496, 8: 3009, 9: 2263, 10: 4732}
test 标签分布: {1: 5022, 2: 2302, 3: 2541, 4: 2635, 7: 2307, 8: 2850, 9: 2344, 10: 4999}
加载了 25000 训练样本, 25000 测试样本
📊 数据划分...
执行可复现的数据划分（带类别检查）...
数据划分: 训练集=32000, 验证集=8000, 测试集=10000
训练集 类别分布: {1: 6478, 2: 2935, 3: 3175, 4: 3412, 7: 3074, 8: 3750, 9: 2948, 10: 6228}
验证集 类别分布: {1: 1620, 2: 734, 3: 794, 4: 853, 7: 768, 8: 937, 9: 737, 10: 1557}
测试集 类别分布: {1: 2024, 2: 917, 3: 992, 4: 1066, 7: 961, 8: 1172, 9: 922, 10: 1946}
🧹 清洗文本数据...
  文本清洗 5000/32000 个样本
  文本清洗 10000/32000 个样本
  文本清洗 15000/32000 个样本
  文本清洗 20000/32000 个样本
  文本清洗 25000/32000 个样本
  文本清洗 30000/32000 个样本
✅ 文本清洗完成
  文本清洗 5000/8000 个样本
✅ 文本清洗完成
  文本清洗 5000/10000 个样本
✅ 文本清洗完成
📚 构建词汇表...
构建词汇表...
词汇表构建完成: 30000 个token
词汇表覆盖率: 97.51%
🔢 向量化数据...
  已向量化 5000/32000 个样本
  已向量化 10000/32000 个样本
  已向量化 15000/32000 个样本
  已向量化 20000/32000 个样本
  已向量化 25000/32000 个样本
  已向量化 30000/32000 个样本
  已向量化 5000/8000 个样本
  已向量化 5000/10000 个样本
🗂️ 创建数据集...
开始保存多分类(1-10星

C:\Users\f5428489\AppData\Local\Temp\ipykernel_4412\2919244761.py:204: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(data_path)
C:\Users\f5428489\AppData\L